In [5]:
import jieba
import pandas as pd
from gensim.models import Word2Vec
import numpy as np

In [6]:
df_all=pd.read_json('./jieba_top40.json',lines=True)
del df_all['_id']
df_all.head()

,書名,ISBN,jieba_cut
0,詭圖：地圖歷史上最偉大的神話、謊言和謬誤（精裝）,9789869590235,"全彩,高度,教授,博物,荒野,副教授,排行榜,未知,困境,揭露,大地,征服,探索,主人,整理..."
1,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,"世界史,神級,考前,年表,死背,事件,高分,故事,好看,特色,分章,整理,不用,瞄過,狂銷,..."
2,忘了自己是動物的人類：重思生命起源的歷史與身而為人的意義,9789864779376,"認為,身為,台灣,起源,思考,星球,人性,狂想,故事,義荷,葡俄西,視為,將人,將身,作為,..."
3,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,..."
4,2021世界年鑑,9789869278799,"疫情,全球,大事,反送,肆虐,入主,肺炎,世界,深入,整理,香港,全面,控制,北京"


In [7]:
userstar_df=pd.read_csv('./userstar.csv')
del userstar_df['USERSTAR']
del userstar_df['CONTENT']
del userstar_df['USER']
del userstar_df['words']
del userstar_df['書名']
userstar_df.head()

,ISBN
0,9789863844433
1,9789570856712
2,9789570856712
3,9789868271272
4,9789868271272


In [8]:
userstar_df.shape

(57641, 1)

In [9]:
df_all['ISBN'] =df_all['ISBN'].astype('str') 
userstar_df['ISBN'] = userstar_df['ISBN'].astype('str') 
df=pd.merge(df_all,userstar_df,on=['ISBN'],how='inner')
df.head()

,書名,ISBN,jieba_cut
0,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,"世界史,神級,考前,年表,死背,事件,高分,故事,好看,特色,分章,整理,不用,瞄過,狂銷,..."
1,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,..."
2,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,..."
3,跟史家一起創作：近代史學的閱讀方法與寫作技藝,9789865080747,"史家,人文,文史,受用,案例,江湖,方法,思考力,故事,工具,文史系,實作力,課綱,人能,能..."
4,歷史學柑仔店 1,9789869865661,"柑仔店,大眾,店小二,公眾,文章,史料,思考,巷仔口,共筆,能店,人客,性共筆,青壯輩,民眾..."


In [10]:
df=df.drop_duplicates(subset=['ISBN','jieba_cut'],ignore_index=True)
df

,書名,ISBN,jieba_cut
0,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,"世界史,神級,考前,年表,死背,事件,高分,故事,好看,特色,分章,整理,不用,瞄過,狂銷,..."
1,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,..."
2,跟史家一起創作：近代史學的閱讀方法與寫作技藝,9789865080747,"史家,人文,文史,受用,案例,江湖,方法,思考力,故事,工具,文史系,實作力,課綱,人能,能..."
3,歷史學柑仔店 1,9789869865661,"柑仔店,大眾,店小二,公眾,文章,史料,思考,巷仔口,共筆,能店,人客,性共筆,青壯輩,民眾..."
4,西方憑什麼：五萬年人類大歷史，破解中國落後之謎,9789868271272,"書獎,地理,病菌,中西,前人,女王,衰退,征服,冰帽,主宰世界,力排,旁徵博引,認為,眾議,..."
...,...,...,...
28775,城市獵人35,9789571305943,"爆笑,生本,剛縣,賞為,手塚治虫,賞以,獠成,成版,久美,名噪,全本,卡通,投稿,男主角,女..."
28776,怪奇檔案全,9789573464747,"野出志,巨匠,怪病,留下"
28777,阿保的童話,9789576930041,"開啟,寄情,映象,孩童,有情,意境,成年人,天地,天空,故事,得到"
28778,小鎮人家,9789576930058,"散文,美代,失鏡,人家,寓言,憎恨,哲理,想像,有如,事件"


In [11]:
# 把str轉成list 
df['jieba_list']=df.jieba_cut.apply(lambda x:x.split(','))

In [12]:
# 隨機找兩本書
print(df[df.ISBN=='9789865051112'])
print(df[df.ISBN=='9789576930072'])

                    書名           ISBN  \
8551  長生外傳：霸道總裁愛上我 （上）  9789865051112   

                                             jieba_cut  \
8551  霸道,陌生人,好像,初吻,前求,作為,存封,心扉,溜走,自持,敲打,眼神,直至,不成,男人,建立   

                                             jieba_list  
8551  [霸道, 陌生人, 好像, 初吻, 前求, 作為, 存封, 心扉, 溜走, 自持, 敲打, ...  
         書名           ISBN                jieba_cut  \
28779  森林小語  9789576930072  小語,留白,森林,省略,趣味,想像,懂得,文字   

                             jieba_list  
28779  [小語, 留白, 森林, 省略, 趣味, 想像, 懂得, 文字]  


In [13]:
# 結合全部jieba_list
jieba_list=df.jieba_list.tolist()

In [ ]:
# 建立模型
# model = Word2Vec(jieba_list, vector_size=100,min_count=1,workers=7, window =3, sg = 1) #size=50

In [14]:
# 存出 存取模型
# model = Word2Vec.save("word2vec_book_gensim.model")
model = Word2Vec.load("word2vec_book_gensim.model")

In [15]:
# 找書本的斷字 
exp_sent_1 = df.iloc[0,3]
print('1',exp_sent_1)
exp_sent_2 = df.iloc[1,3]
print('2,',exp_sent_2)

1 ['世界史', '神級', '考前', '年表', '死背', '事件', '高分', '故事', '好看', '特色', '分章', '整理', '不用', '瞄過', '狂銷', '敲頭', '記年', '山崎', '金三角', '卷末', '速成', '概貌', '文化史', '篇章', '表格', '穿插', '尖叫', '排行榜', '主角', '剔除', '航海', '影片', '走上', '打破', '文字', '理由', '吸收', '掌握', '推出', '不得']
2, ['狐狸', '甘迺迪', '掌握', '學程', '失準', '政策', '心法', '探看', '政治', '全局', '跨越', '同事', '思考', '方式', '原本', '整合', '人才', '世界', '需知', '認為', '眾人', '不選', '秘辛', '員們', '記裡', '現握', '登思', '策略', '手段', '德利', '力量', '蠢事', '匈牙利人', '步出', '身兼', '造成', '指出', '配角', '俯瞰', '面相']


In [16]:
# 統一維度
vec_sent_1_w2v = np.zeros(100)
vec_sent_2_w2v = np.zeros(100)
for w in exp_sent_1:
    vec_sent_1_w2v += model.wv[w]
print(vec_sent_1_w2v)
for w in exp_sent_2:
    vec_sent_2_w2v += model.wv[w]
print(vec_sent_2_w2v)

[ 9.54168029e-02  4.96503407e+00  2.64362710e+00  2.13015763e+00
 -3.08910247e+00 -7.38902028e+00 -8.58277542e+00  1.70086022e+01
  4.35057039e+00 -1.07814195e+01 -1.67329928e+01 -2.23582894e+01
  1.50133443e+00  6.65915843e+00 -2.62042817e+00 -2.26242904e+00
 -5.08550717e-01 -2.09333335e+01  1.92623323e+01 -2.79792390e+01
  8.91637946e+00  2.00104690e+01 -3.95812394e+00 -5.85958340e-01
  3.52942318e+00  4.36154133e+00  1.08154982e+01 -1.18723474e+01
 -1.48261316e+01 -3.14836387e+00  9.38826825e+00  5.84365351e+00
 -3.34222901e+00 -1.01131454e+01 -4.58578328e+00  2.04849079e+01
  7.11896866e+00 -1.08871743e+00 -1.61512283e+01 -3.42208325e+01
  1.38611034e+01 -2.51095852e+01 -1.87203932e+01  3.49760122e+00
  1.76250118e+01  9.37343292e+00 -1.70073461e+01 -4.27527948e+00
 -2.67692402e-01 -4.14607522e+00  3.27006242e+00 -1.05696823e+01
 -1.89117550e+01 -6.60616821e+00 -1.34384444e+01  3.50317066e-02
  4.72554795e+00 -4.69565637e+00 -2.36726954e+01 -6.95934679e-01
  1.38691031e+01 -2.30710

In [17]:
# 兩本書的相似度
w2v_similarity = np.dot(vec_sent_1_w2v, vec_sent_2_w2v) / (np.linalg.norm(vec_sent_1_w2v)*np.linalg.norm(vec_sent_2_w2v))
print('similarity of Word2Vec+: ', w2v_similarity)

similarity of Word2Vec+:  0.9453857982363106


In [27]:
import time
from operator import itemgetter
from pymongo import MongoClient
import json
client = MongoClient()
db = client.similar
coll = db.similar
# coll.drop()
start=time.time()

for i in range(2209,2211):
    similar=[]
    for j in range(len(df)):
        exp_sent_1 = df.iloc[i,3]
        exp_sent_2 = df.iloc[j,3]
        vec_sent_1_w2v = np.zeros(100)
        vec_sent_2_w2v = np.zeros(100)
        for w in exp_sent_1:
            vec_sent_1_w2v += model.wv[w]
        for w in exp_sent_2:
            vec_sent_2_w2v += model.wv[w]
        w2v_similarity = np.dot(vec_sent_1_w2v, vec_sent_2_w2v) / (np.linalg.norm(vec_sent_1_w2v)*np.linalg.norm(vec_sent_2_w2v))
        similar.append((df.iloc[j,1],w2v_similarity))
    similar.sort(key=itemgetter(1), reverse=True)
    # 存入資料庫
    kv={"ISBN":df.iloc[i,1],"list":[similar[1][0],similar[2][0],similar[3][0]]}
    coll.insert([kv])
    print(df.iloc[i,1])
    end=time.time()
print(f"總共花費：{int(end-start)} 秒")

C:\Users\Tibame\AppData\Local\Temp/ipykernel_5436/2322567822.py:27: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  coll.insert([kv])


9789866571145
9789866662898
總共花費：15 秒


In [25]:
df[df['ISBN']=='9789866571145']

,書名,ISBN,jieba_cut,jieba_list
2209,天真的人類學家 小泥屋筆記,9789866571145,"教授,巴利,幽默感,洞察力,非洲,利幾,之林,利以,啟發,趣味十足,不起,巴樂馬,提絲,所會...","[教授, 巴利, 幽默感, 洞察力, 非洲, 利幾, 之林, 利以, 啟發, 趣味十足, 不..."


In [26]:
df[2209:].head()

,書名,ISBN,jieba_cut,jieba_list
2209,天真的人類學家 小泥屋筆記,9789866571145,"教授,巴利,幽默感,洞察力,非洲,利幾,之林,利以,啟發,趣味十足,不起,巴樂馬,提絲,所會...","[教授, 巴利, 幽默感, 洞察力, 非洲, 利幾, 之林, 利以, 啟發, 趣味十足, 不..."
2210,菁英的反叛,9789866662898,"菁英,民主,社群,分子,反叛,理想,操弄,認為,群眾,問化,民眾,回歸,思想,拔擢,迷失,洪...","[菁英, 民主, 社群, 分子, 反叛, 理想, 操弄, 認為, 群眾, 問化, 民眾, 回..."
2211,自己幹文化-派對與革命,9789572938621,"文化,上山下海,開啟,舞客,宣示,制度化,反省,景象,收集,角色,禁止,模式,公路,作者,限...","[文化, 上山下海, 開啟, 舞客, 宣示, 制度化, 反省, 景象, 收集, 角色, 禁止..."
2212,知識人與中國文化的價值,9789571346427,"文化,人文,儒家,超越,有士,形塑,人求,旁徵博引,選賢,唯才,建請,禮易,規臬,試說,可辱...","[文化, 人文, 儒家, 超越, 有士, 形塑, 人求, 旁徵博引, 選賢, 唯才, 建請,..."
2213,死也要上報!,9789866852015,"死亡,作家,天使,整份,奇行,怪癖,之能事,化身,世界,八卦,英美,格式,捕捉,面貌,代表性...","[死亡, 作家, 天使, 整份, 奇行, 怪癖, 之能事, 化身, 世界, 八卦, 英美, ..."
